# The Goal
The goal of this project is making a sentiment analysis on comments of the most popular online food order and delivery company of Turkey which is Yemeksepeti in Turkish Language. In the beginning of this study, I have read a lot of articles about NLP studies about English language and also some about Turkish Language. My main finding about this research was Turkish Language suffers scarcity of these type of NLP works. The structure of Turkish and English is really different and for that reason, the preprocessing steps should be different, too. So I decided to go deeper in NLP, understand the underlying logic behind it and what should be the correct way of preprocessing steps for sentiment analysis in Turkish. Let's begin.

In [ ]:
pip install JPype1

In [ ]:
import re, os, pickle
from collections import Counter
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from string import punctuation, digits
import itertools
import pandas as pd
import numpy as np
from pathlib import Path
import nltk
from nltk.corpus import stopwords
import re
from string import punctuation, digits
import jpype as jp
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import scale
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras

# Preprocess Functions

In [ ]:
def lower(text):
    text=re.sub("İ","i",text)
    text = text.lower()
    return text

In [ ]:
def resubComma(texts):
    texts= re.sub(","," ",texts)
    return texts

In [ ]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
print(stop_word_list)

The lower and resubComma functions are pretty self-explanatory.

Stopwords are words that are used too often in sentences but they don't change the meaning of sentence. Because of this, we need to remove them all from our sentences. I used NLTK library's stopwords list to specify them.

In [ ]:
def replace_emoticon(word):
    check_pos = re.findall(r'(?::\)|:-\)|=\)|:D|:d|<3|\(:|:\'\)|\^\^|;\)|\(-:)', word)
    check_neg = re.findall(r'(:-\(|:\(|;\(|;-\(|=\(|:/|:\\|-_-|\):|\)-:)', word)
    if check_pos:
        #word = ":)"
        word = "SMILEYPOSITIVE"
    elif check_neg:
        #word = ":("
        word = "SMILEYNEGATIVE"
    return word

In [ ]:
def vanish_punc(text):
    regex = re.compile('[%s]' % re.escape(punctuation))
    text = regex.sub(' ', text)
    return text

We want to remove all punctuations because they don't have a direct meaning at all. But before, maybe we can use some of the punctuations in favor of us. The replace_emoticon function is doing that for us and it converts some emoticons to words. It is really game changing preprocessing step especially when it comes to sentiment analysis, because they directly filled with emotions. 

In [ ]:
def vanish_digits(text):
    text=text.strip()
    vanish_digits = str.maketrans('', '', digits)
    text=text.translate(vanish_digits)
    return text

In some cases, digits can be useful. However, in this project, I don't think that they are worth to deal with. So I remove them all.

In [ ]:
def dup_vanish(s1):
     return (''.join(i for i, _ in itertools.groupby(s1)))

This step is for excited and angry writers who don't care about spelling rules. For example, think about this kind of comment. "Yemeeeek harikaydıııı." We can not expect this kind of comment can be interpreted just regularly by our neural nets. So we need to make it a regular comment. Like, "Yemek harikaydı". The function named dup_vanish maks exactly this. However, there is a trade-off. Because of we vanish all the duplicates in words, some regular words are become misspelled. Like, "Yemek mükemmeldi.".After dup_vanish function it becomes "Yemek mükemeldi" which is not as we wanted. In the next steps, we are trying to solve this little problem

In [ ]:
def reverse(s): 
    if len(s) == 0: 
        return s 
    else: 
        return reverse(s[1:]) + s[0] 

def checkOpennes(word):
    vowels=['a','e','i','ı','o','ö','u','ü']
    open_vowels=['e','i','ü','ö']
    close_vowels=['a','ı','o','u']
    for i in range(len(word)):
        if reverse(word)[i] in vowels:
            if reverse(word)[i] in open_vowels:
                return True
            else:
                return False
        else:
            continue
def PresentCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if reverse(word)[i] in ei:
                return 'ei'
            elif reverse(word)[i] in üö:
                return 'üö'
            elif reverse(word)[i] in aı:
                return 'aı'
            elif reverse(word)[i] in uo:
                return 'uo'
            else:
                continue
    
def StartCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if (word)[i] in ei:
                return 'ei'
            elif (word)[i] in üö:
                return 'üö'
            elif (word)[i] in aı:
                return 'aı'
            elif (word)[i] in uo:
                return 'uo'
            else:
                continue

In [ ]:
WORDS = dict()

spell_folder = Path(r"..//input//turkish-spell-check")

def words(text): return re.findall(r'\w+', text.lower())
with open(os.path.expanduser(Path(spell_folder/ "big2.txt")), "r", encoding = 'utf-8') as f:
    for line in f:
        splitted = line.split()
        WORDS[splitted[0]] = int(splitted[1])

For spell correction dictionary I have used Orhan Bilgin's frequency dictionary. It includes lots of word and these frequences and selected the most probable word always.

In [ ]:
def replaceall(s, n,a):
    occurence = s.count(n)
    alt = []
    temp = s
    for i in range(occurence):
        temp2 = temp
        for j in range(i,occurence):
            temp2 = temp2.replace(n,a,1)
            alt.append(temp2)
        temp = temp.replace(n,"!",1)
    for i in range(len(alt)):
        alt[i] = alt[i].replace("!",n)

    return alt

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    if  word in WORDS.keys():
        number = WORDS[word]
    else:
        number = 1
    if number == 0:
        number = 1
    return number / N

def correction(word):
      return max(candi(word), key=P)

def candi(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefgğhıijklmnoöprsştuüvyzw'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
 
    sp     = replaceall(word,'ı','i')
    sp2     = replaceall(word,'u','ü')
    sp3    = replaceall(word,'o','ö')
    sp4     = replaceall(word,'g','ğ')
    sp5     = replaceall(word,'c','ç')
    sp6     = replaceall(word,'s','ş')
    sp7     = replaceall(word,'i','ı')
    sp8     = replaceall(word,'ö','o')
    sp9     = replaceall(word,'ş','s')
    sp10     = replaceall(word,'ğ','g')
    sp11     = replaceall(word,'ç','c')
    sp12     = replaceall(word,'ü','u')
    specials=[]
    specials.extend(sp)
    specials.extend(sp2)
    specials.extend(sp3)
    specials.extend(sp4)
    specials.extend(sp5)
    specials.extend(sp6)
    specials.extend(sp7)
    specials.extend(sp8)
    specials.extend(sp9)
    specials.extend(sp10)
    specials.extend(sp11)
    specials.extend(sp12)
    return set(deletes+transposes+replaces+inserts+specials)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def print_diff(word, s):
    if not word == s:
        print(word + " --> " + s)
counter = 0


The functions in the up cell is some kind of algorithm that tries to correct spelling. It tries many possible combinations of the word and then search these candidate words in Orhan Bilgin's dictionary and select the most probable one. 

In [ ]:
ZEMBEREK_PATH = r'../input/zemberek/zemberek-full.jar'
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

# The Lemmatizer

In English, there is very limited ways of making a sentence negative.(Simply, "not"). However, in Turkish there are many ways to do that like word "değil", "-me" afformative to the verb, word "yok" and many more ways. We should not remove this words and afformatives remark that negativity of sentence since we are making sentiment analysis. So I tried to this lemmatizer without removing this words and afformatives.

It is very basic and not optimized lemmatizer. Thank you for making this Zemberek library!

In [ ]:

TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
TurkishSpellChecker = jp.JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = jp.JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = jp.JClass('java.nio.file.Paths')
# Get the path to the (baseline) lookup files
lookupRoot = Paths.get(r'../input/zemberek2/normalization')
# Get the path to the compressed bi-gram language model
lmPath = Paths.get(r'../input/zemberek3/lm.2gram.slm')
morphology = TurkishMorphology.createWithDefaults()
# Initialize the TurkishSentenceNormalizer class
# Instantiate the morphology class with the default RootLexicon
morph = TurkishMorphology.createWithDefaults()

# Instantiate the spell checker class using the morphology instance
spell = TurkishSpellChecker(morph)

#normalizer = TurkishSentenceNormalizer(morphology, lookupRoot, lmPath)

In [ ]:
morphology = TurkishMorphology.createWithDefaults()


def lemmatizer(word,texts):
        wordList=[]
        wordList = re.sub("[^\w]", " ",  texts).split()
        if '�' in word:
            return 'question'
        pos=wordList.index(word)
        if word=="SMILEYPOSITIVE":
            return word
        if word=="SMILEYNEGATIVE":
            return word
        sakin=''
        word=correction(word)
        if len(wordList)-pos>3 and pos>2:
            for i, kelime in enumerate(wordList[pos-3:pos+4]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos>5:
            for i, kelime in enumerate(wordList[pos:pos+5]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos<=5:
            for i, kelime in enumerate(wordList[pos:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)-pos<1 and pos>3:
            for i, kelime in enumerate(wordList[pos-3:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)<3:
            for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        else:
             for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        results = morphology.analyze(word)
        lemma=[]
        form=[]
        l=[]
        m=[]
        for i, result in enumerate(results):
            form.append(str(result.formatLong()))
            lemma.append(result.getLemmas()[0])
        if len(lemma)>1:
                analysis = morphology.analyzeSentence(sakin)
                results = morphology.disambiguate(sakin, analysis).bestAnalysis()
                for i, result in enumerate(results):
                        l.append(result.getLemmas()[0])
                        m.append(result.formatLong())
                for i in range(len(m)):
                    for j in range(len(form)):
                        if m[i]==form[j]:
                            lema=lemma[j]
                            if lema=='değil':
                                return 'değil'
                            if 'Neg' in form[j] or 'WithoutHavingDoneSo' in form[j] or 'Unable' in form[j]:
                                if checkOpennes(word):
                                    return lema+'me'
                                else:
                                    return lema+'ma'
                            if 'Without' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'siz'
                                elif PresentCheck(word)=='aı':
                                    return lema+'sız'
                                elif PresentCheck(word)=='uo':
                                    return lema+'suz'
                                else:
                                    return lema+'süz'
                            if 'With' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'li'
                                elif PresentCheck(word)=='aı':
                                    return lema+'lı'
                                elif PresentCheck(word)=='uo':
                                    return lema+'lu'
                                else:
                                    return lema+'lü'
                            else:
                                return lema
                    else:
                        continue
        elif len(lemma)==1:
            if lemma[0]=='değil':
                return lemma[0]
            if 'Neg' in form[0] or 'WithoutHavingDoneSo' in form[0] or 'Unable' in form[0]:
                 if checkOpennes(word):
                    return lemma[0]+'me'
                 else:
                    return lemma[0]+'ma'
            elif 'Without' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'siz'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'sız'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'suz'
                else:
                    return lemma[0]+'süz'
            elif 'With' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'li'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'lı'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'lu'
                else:
                    return lemma[0]+'lü'
            else:
                return lemma[0]
        else:
            return word


In [ ]:
def clean_messages(texts):
    texts=lower(texts)
    texts=resubComma(texts)
    tokens=WPT.tokenize(texts)
    text = [token for token in tokens if token not in stop_word_list]
    for i, word in enumerate(text):
        text[i]=dup_vanish(vanish_digits(vanish_punc(replace_emoticon((word)))))
    text = list(filter(None,text))
    metin=' '.join(text)
    for i, word in enumerate(text):
         text[i] = lemmatizer(word,metin)
    test=' '.join(text)
    return test

# Data Manipulation

In [ ]:
data=pd.read_csv(r"../input/yemeksepeti/yemeklerin_sepeti.csv")

In [ ]:
data=data.dropna()
data = data.drop_duplicates(subset=['yorum'])
data=data.reset_index()
data=data.drop('index',axis=1)

In [ ]:
for i in range(len(data.hiz)):
    data.hiz[i]=re.sub("\D", "", data.hiz[i])
for i in range(len(data.servis)):
    data.servis[i]=re.sub("\D", "", data.servis[i])
for i in range(len(data.lezzet)):
    data.lezzet[i]=re.sub("\D", "", data.lezzet[i])
       

In [ ]:
data['hiz'] = data['hiz'].astype(int)
data['servis'] = data['servis'].astype(int)
data['lezzet'] = data['lezzet'].astype(int)

In [ ]:
data.loc[data['hiz']<4, 'hiz'] = 0
data.loc[data['hiz'] >7, 'hiz'] = 1
data.loc[data['lezzet']<4, 'lezzet'] = 0
data.loc[data['lezzet'] >7, 'lezzet'] = 1
data.loc[data['servis']<4, 'servis'] = 0
data.loc[data['servis'] >7, 'servis'] = 1



In [ ]:
data=data[(data['hiz']<2) & (data['servis']<2) & (data['lezzet']<2)]

In [ ]:
data_positive=data[(data['hiz']==1) & (data['servis']==1) & (data['lezzet']==1)]
data_negative=data[(data['hiz']==0) & (data['servis']==0) & (data['lezzet']==0)]

In [ ]:
data_positive=data_positive[:600]

In [ ]:
data=pd.concat([data_positive,data_negative],axis=0)
data=data.drop_duplicates()

Let's examine some of the comments.

In [ ]:
print(data.head())
print(data.tail())

In [ ]:
data['isPositive']=np.where(((data['hiz']==1) & (data['servis']==1) & (data['lezzet']==1)),1,0)
data=data.drop(["hiz","servis","lezzet"],axis=1)

In [ ]:
new=[]
for message in data['yorum']:
      new.append(clean_messages(message))

# The Model

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new, data['isPositive'], test_size=0.2 ,random_state=1)

Let's examine some of preprocessed comments.

In [ ]:
print(x_train[:5])
print(x_train[-5:])

# Word Embeddings in Turkish

I used https://github.com/akoksal/Turkish-Word2Vec this rep and this pretrained model to convert words to vectors.

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format(r"..//input//wordvec//trmodel", binary=True)

In [ ]:
def buildWordVector(text, size=400):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += word_vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec


In [ ]:
tokensTrain=[]
for i in range(0,len(x_train)):
    tokensTrain.append(WPT.tokenize(x_train[i]))

train_vecs = np.concatenate([buildWordVector(z) for z in tokensTrain])
train_vecs=scale(train_vecs)

tokensTest=[]
for i in range(0,len(x_test)):
    tokensTest.append(WPT.tokenize(x_test[i]))

test_vecs = np.concatenate([buildWordVector(z) for z in tokensTest])
test_vecs=scale(test_vecs)


In [ ]:
test_vecs = np.reshape(test_vecs, (test_vecs.shape[0], 1,test_vecs.shape[1]))
train_vecs  = np.reshape(train_vecs , (train_vecs.shape[0],1,train_vecs.shape[1]))

Reshaped the datas to fit LSTM model.

In [ ]:
model = Sequential()
model.add(LSTM(200,input_shape=(1,train_vecs.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))

my_optimizer = tf.keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=my_optimizer, metrics=['accuracy'])
model.summary()

mc = keras.callbacks.ModelCheckpoint("../working/best_val_loss", monitor='val_loss', verbose=0, save_best_only=True, mode='min',
                                     save_weights_only=True)

es = keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)

In [ ]:
model.fit(train_vecs[:-300], y_train[:-300],
          epochs=600,batch_size=2, verbose=2,
         validation_data=(train_vecs[-300:], y_train[-300:]),
          callbacks=[es,mc])

To avoid overfitting, we saved our model with best validation loss and now we will load it.

In [ ]:
(model.load_weights("../working/best_val_loss"))

In [ ]:
y_pred=model.predict(test_vecs,
                     batch_size=1,
                     verbose=1, steps=None)

In [ ]:
y_pred=y_pred>0.5


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=y_pred,y_true=y_test)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])


In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
print('Accuracy: %f' % accuracy)

Lastly' let's inspect our wrong predictions.

In [ ]:
y_test = y_test.reset_index(drop = True)
y_test=np.reshape(y_test,(len(y_test,)))
y_pred=np.reshape(y_pred,(len(y_pred,)))

x_test = pd.Series(x_test)
y_test=pd.Series(y_test)
y_pred=pd.Series(y_pred)
inspection = pd.concat([x_test, y_test, y_pred], axis=1)

inspection.columns=['Yorum', 'Test','Predictions']

wrong_predictions = inspection[inspection["Predictions"] != inspection["Test"]]
print(wrong_predictions)

# Analysis Of Predictions


These comments might seem meaningless because they are preprocessed comments and we can say that the wrong predictions are likely be more meaningless than true predictions. If we had a perfect word2vec model, we didn't have to make a complex lemmatizer like this. As I see, many wrong predictions caused by one positive and one negative word side by side that changes meaning of sentence. For example, "iyi değil" or "güzel değil". Again, this is caused by our pretrained word2vec model is not perfect. In my opinion, some comments are inconsistent with points. For instance, in this trial, the comment is "sen ye eski sevgili gel akıl SMILEYNEGATIVE" If I were the model and evaluate this comment I would give negative, too. However, commentor gave high points.

Thank you for reading.
Best regards :)

Ege Baştürk